In [2]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import pandas as pd


# model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
# if torch.cuda.is_available():
#     model.cuda()

# def predict_pair(text1, text2):
#     with torch.inference_mode():
#         out = model(**tokenizer(text1, text2, return_tensors='pt').to(model.device))
#         proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
#         return {v: proba[k] for k, v in model.config.id2label.items()}['entailment']

In [ ]:
# import pandas as pd

# df = pd.read_csv('data/raw/mcoco.csv')
# df.caption = df.caption.apply(eval)

In [ ]:
import tqdm
from itertools import permutations


data = []
for _, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    cluster_id = row['image_id']
    for pair in permutations(set(row['caption']), 2):
        try:
            proba = predict_pair(*pair)
        except RuntimeError as e:
            print(e)
            continue
        data.append((cluster_id, *pair, proba))

In [3]:
import pandas as pd

In [ ]:
# df_pairs = pd.DataFrame(data, columns=['cluster_id', 'text1', 'text2', 'proba'])

In [ ]:
# df_pairs.to_csv('mcoco_pairs_entailment.csv', index=False)

In [4]:
df = pd.read_csv('data/mcoco_entailment.csv')

In [5]:
import tqdm

In [6]:
df2 = df.copy()
df2.columns = ['cluster_id', 'text2', 'text1', 'proba']

In [23]:
df = df.merge(df2, on = ['text1', 'text2'])
df = df[['cluster_id_x', 'text1', 'text2', 'proba_x', 'proba_y']]
df.columns = ['cluster_id', 'text1', 'text2', 'ent1', 'ent2']

In [30]:
import numpy as np

dupes = pd.DataFrame(np.sort(df[['text1', 'text2']].values, axis=1),
                     index=df[['text1', 'text2']].index,
                     columns=df[['text1', 'text2']].columns)
df = df[~dupes.duplicated()]

In [58]:
df = df[(df.ent1 > 0.7) & (df.ent2 > 0.7)]

In [98]:
from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()


df = df[~(df.text1.apply(lambda x: ad.is_latin(x)))]
df = df[~(df.text2.apply(lambda x: ad.is_latin(x)))]

In [100]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
from collections import Counter
from razdel import tokenize

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

def match_ents(s1, s2):
    doc1, doc2 = Doc(s1), Doc(s2)
    
    doc1.segment(segmenter)
    doc2.segment(segmenter)
    
    doc1.tag_ner(ner_tagger)
    doc2.tag_ner(ner_tagger)
    
    if Counter([s.type for s in doc1.spans]) != Counter([s.type for s in doc2.spans]):
        return False
    
    numbers1 = set()
    numbers2 = set()
    
    for tok in doc1.tokens:
        if has_numbers(tok.text):
            numbers1.add(tok.text)
    
    for tok in doc2.tokens:
        if has_numbers(tok.text):
            numbers2.add(tok.text)
    
    if numbers1 != numbers2:
        return False
    
    return True
    

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def match_abbr(s1, s2):
    abbrs1 = set()
    abbrs2 = set()
    
    for tok in tokenize(s1):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs1.add(tok.text)
    for tok in tokenize(s2):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs2.add(tok.text)
    if abbrs1 == abbrs2:
        return True
    return False

def match_latin(s1, s2):
    lat1 = set()
    lat2 = set()
    
    for tok in tokenize(s1):
        if ad.islatin(tok.text):
            lat1.add(tok.text)
    for tok in tokenize(s2):
        if ad.islatin(tok.text):
            lat2.add(tok.text)

    if lat1 == lat2:
        return True
    return False

In [101]:
df = df[df.apply(lambda x: match_abbr(x['text1'], x['text2']), axis=1)]

In [102]:
len(df)

18204

In [107]:
import tqdm

ner_data = []
for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    if match_ents(row['text1'], row['text2']):
        ner_data.append(i)

100%|██████████| 18204/18204 [00:41<00:00, 442.19it/s]


In [142]:
df = df[df.index.isin(ner_data)]

In [143]:
df

,cluster_id,text1,text2,ent1,ent2
0,9,"Куча предателей, у которых разная еда.","есть контейнеры, наполненные различными видами...",0.963380,0.979076
351,92,Кусок шоколада на тарелке с салфеткой и вилкой.,шоколадный торт и готовая к употреблению вилка,0.748423,0.802098
355,92,Шоколадное десерт на тарелке с вилкой.,шоколадный торт и готовая к употреблению вилка,0.860718,0.918646
415,110,Женщина резала пиццу вилкой и ножом,Женщина режет пиццу вилкой и ножом.,0.968125,0.984165
721,196,"Стол с морковью, горохом, салатом, кукурузой, ...","Стол, полный еды, такой как горох и морковь, х...",0.962813,0.746812
...,...,...,...,...,...
2268206,581815,В водоеме плавает стайка уток.,Несколько уток плавают вместе в водоеме.,0.962703,0.972966
2268468,581882,"санузел с раковиной, зеркалом и раковиной",В ванной комнате есть раковина и зеркало.,0.948727,0.904250
2268496,581884,Группа людей летающих котят на пляже,"Пляж с летающими котятами, пока люди смотрят.",0.793174,0.965322
2268501,581884,"Пляж с летающими котятами, пока люди смотрят.",Толпа людей летающих котят на песчаном пляже.,0.838597,0.912526


In [151]:
df.to_csv('data/clean/mcoco_clean.tsv', sep='\t', index=False)

In [150]:
k = df.sample(1).iloc[0]
print(k['text1'])
print(k['text2'])

черно-белая кошка и черно-коричнево-белая, сидящая на чемодане
Две белые и черные кошки сидят на сумке для багажа.
